In [10]:
import requests
from datetime import datetime

def fetch_transactions(api_url, access_token, start_time, end_time, limit):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    
    transactions = []
    offset = 0
    
    while True:
        # Build the request URL
        request_url = f"{api_url}/v2/accounts/liquid/transactions"
        params = {
            'start': start_time,
            'end': end_time,
            'limit': limit,
            'offset': offset
        }
        
        # Send the GET request
        response = requests.get(request_url, headers=headers, params=params)
        
        if response.status_code != 200:
            raise Exception(f"Error fetching transactions: {response.status_code} - {response.text}")
        
        data = response.json()
        print(data)
        fetched_transactions = data.get('data', [])
        
        if not fetched_transactions or len(fetched_transactions) < limit:
            # No more transactions or fewer than limit (last batch)
            transactions.extend(fetched_transactions)
            break
        
        # Add fetched transactions to the list
        transactions.extend(fetched_transactions)
        offset += limit
    
    return transactions

# Example usage
if __name__ == "__main__":
    from dotenv import load_dotenv
    import os
    from pyzettle import authenticate
    load_dotenv()
    client_id = os.getenv("client_id")
    api_key = os.getenv("key")
    auth = authenticate.Authenticate(client_id, api_key)
    API_URL = "https://finance.izettle.com"  # Replace with the actual base URL for the Finance API
    ACCESS_TOKEN = auth.access_token  # Replace with your OAuth access token
    START_TIME = "2023-09-20T00:00:00-00:00"
    END_TIME = "2023-09-29T00:00:00-00:00"
    LIMIT = 3
    
    transactions = fetch_transactions(API_URL, ACCESS_TOKEN, START_TIME, END_TIME, LIMIT)
    
    for txn in transactions:
        print(txn)


[{'timestamp': '2023-09-26T00:25:28.592+0000', 'amount': -18, 'originatorTransactionType': 'PAYMENT_FEE', 'originatingTransactionUuid': '13085b76-5be0-11ee-8714-ed8f4239fc90'}, {'timestamp': '2023-09-26T00:25:28.591+0000', 'amount': 1000, 'originatorTransactionType': 'PAYMENT', 'originatingTransactionUuid': '13085b76-5be0-11ee-8714-ed8f4239fc90'}, {'timestamp': '2023-09-26T00:25:26.364+0000', 'amount': -88, 'originatorTransactionType': 'PAYMENT_FEE', 'originatingTransactionUuid': '6938320a-5be0-11ee-8695-0b6bf633fdf1'}]


AttributeError: 'list' object has no attribute 'get'

In [4]:
import requests
from typing import List, Dict

def fetch_transactions(
    base_url: str, 
    access_token: str, 
    account_type: str, 
    start_time: str, 
    end_time: str, 
    limit: int = 100, 
    include_transaction_types: List[str] = None
) -> List[Dict]:
    """
    Fetch transactions from a merchant's Zettle account.

    Args:
        base_url (str): The base URL of the Zettle API.
        access_token (str): The OAuth access token.
        account_type (str): The account type ('liquid' or 'preliminary').
        start_time (str): The start time in ISO 8601 format.
        end_time (str): The end time in ISO 8601 format.
        limit (int): Number of transactions per request (default 100).
        include_transaction_types (List[str]): Types of transactions to include.

    Returns:
        List[Dict]: A list of transactions.
    """
    transactions = []
    offset = 0

    while True:
        url = f"{base_url}/v2/accounts/{account_type}/transactions"
        params = {
            'start': start_time,
            'end': end_time,
            'limit': limit,
            'offset': offset
        }

        if include_transaction_types:
            for t in include_transaction_types:
                params[f'includeTransactionType[]'] = t
        
        headers = {
            'Authorization': f'Bearer {access_token}',
            'Content-Type': 'application/json'
        }
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            raise ValueError(f"API request failed with status code {response.status_code}. Response: {response.text}")
        
        data = response.json()
        transactions.extend(data.get('data', []))
        
        if len(data.get('data', [])) < limit:
            break
        
        offset += limit
    
    return transactions

# Example usage:
if __name__ == "__main__":
    from dotenv import load_dotenv
    import os
    from pyzettle import authenticate
    load_dotenv()
    client_id = os.getenv("client_id")
    api_key = os.getenv("key")

    auth = authenticate.Authenticate(client_id, api_key)
    
    BASE_URL = "https://api.zettle.com"
    ACCESS_TOKEN = auth.full_token
    ACCOUNT_TYPE = "liquid"
    START_TIME = "2023-09-20T00:00:00+00:00"
    END_TIME = "2023-09-29T00:00:00+00:00"
    LIMIT = 3
    INCLUDE_TRANSACTION_TYPES = ['PAYMENT', 'PAYMENT_FEE']
    
    transactions = fetch_transactions(
        base_url=BASE_URL,
        access_token=ACCESS_TOKEN,
        account_type=ACCOUNT_TYPE,
        start_time=START_TIME,
        end_time=END_TIME,
        limit=LIMIT,
        include_transaction_types=INCLUDE_TRANSACTION_TYPES
    )
    
    print(transactions)


ConnectionError: HTTPSConnectionPool(host='api.zettle.com', port=443): Max retries exceeded with url: /v2/accounts/liquid/transactions?start=2023-09-20T00%3A00%3A00%2B00%3A00&end=2023-09-29T00%3A00%3A00%2B00%3A00&limit=3&offset=0&includeTransactionType%5B%5D=PAYMENT_FEE (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000240507E91E0>: Failed to resolve 'api.zettle.com' ([Errno 11001] getaddrinfo failed)"))